# Super Bowl Soda Promotion

A planner is preparing for the annual Super Bowl soda promotion. Certain lines of soda will go on sale, with others remaining at their standard price. The objective is to maximize the total sales, while obeying specified KPI restrictions. In addition, cannibalization should be avoided by limiting the number of sales that are applied to the same product family.

At his disposal is a set of historical records from which inferences can be drawn as to how price, weather, holiday status, and preceeding sales effect future demand. By applying machine learning techniques, we can apply the historical data against the current set of similar data to predict the likely demand for each product/price pair. This data, along with product family and KPI information, can then be fed to an optimization problem.

# Machine Learning Section
### Load and explore historical data

In [1]:
import pandas
from pandas import DataFrame, get_dummies
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import BaggingRegressor
from sklearn import model_selection

In [2]:
df_hist = pandas.read_excel("soda_sales_historical_data.xlsx")

In [3]:
df_hist[:5]

,Product,Sales,Cost Per Unit,Easter Included,Super Bowl Included,Christmas Included,Other Holiday,4 Wk Avg Temp,4 Wk Avg Humidity,Sales M-1 weeks,Sales M-2 weeks,Sales M-3 weeks,Sales M-4 Weeks,Sales M-5 weeks
0,11 Down,51.9,1.6625,No,No,Yes,No,80.69,69.19,17.0,22.4,13.5,14.5,28.0
1,Alpine Stream,55.8,2.2725,No,No,Yes,No,80.69,69.19,2.4,2.2,2.0,1.4,0.5
2,Bright,3385.6,1.3475,No,No,Yes,No,80.69,69.19,301.8,188.8,101.4,81.6,213.8
3,Crisp Clear,63.5,1.6600,No,No,Yes,No,80.69,69.19,73.8,69.4,72.8,75.4,57.4
4,Popsi Kola,181.1,1.8725,No,No,Yes,No,80.69,69.19,23.1,22.6,22.1,19.9,23.2


In [4]:
len(df_hist)

596

### Experiment (aka Cross Fold Validation)

Here, we use cross-fold validation to determine which ML algorithm is most appropriate for the historical data.

In [5]:
categorical_columns = ['Product','Easter Included','Super Bowl Included', 'Christmas Included',
                       'Other Holiday']

In [6]:
df_hist = get_dummies(df_hist, prefix={k:"dmy_%s"%k for k in categorical_columns},
                      columns = list(categorical_columns))

In [7]:
experiments = {"Algorithm":["Ordinary Least Squares", "Regression Tree", "Random Forest", 
                            "Bagging"], 
               "Objects" : [LinearRegression(), DecisionTreeRegressor(), 
                            RandomForestRegressor(), BaggingRegressor()], 
               "Results":[]}

In [8]:
for o in experiments["Objects"]:
    experiments["Results"].append(
        model_selection.cross_val_score(o, y=df_hist["Sales"], 
                                        X=df_hist.drop("Sales", axis=1)).mean())

In [9]:
DataFrame(experiments).drop("Objects", axis=1).set_index("Algorithm")

,Results
Algorithm,
Ordinary Least Squares,0.446870
Regression Tree,0.749361
Random Forest,0.866699
Bagging,0.778943


### Fit and Predict

The Super Bowl promotion data table contains all product/pricing pairs, along with the current data for the historical schema.

In [10]:
fitted = RandomForestRegressor().fit(y=df_hist["Sales"], X=df_hist.drop("Sales", axis=1))

In [11]:
df_superbowl_original = pandas.read_excel("super_bowl_promotion_data.xlsx")
df_superbowl = get_dummies(df_superbowl_original, prefix={k:"dmy_%s"%k for k in categorical_columns},
                      columns = list(categorical_columns))
assert "Sales" not in df_superbowl.columns 
assert {"Sales"}.union(df_superbowl.columns).issubset(set(df_hist.columns))
len(df_superbowl)

36

We shouldn't be surprised that the much larger historical data table has categorical selections that don't appear in the Carnivale promotion table.

In [12]:
for fld in set(df_hist.columns).difference(df_superbowl.columns, {"Sales"}):
    assert fld.startswith("dmy_")
    df_superbowl[fld] = 0

In [13]:
predicted = fitted.predict(df_superbowl)

In [14]:
forecast_sales = df_superbowl_original[["Product", "Cost Per Unit"]].copy()
forecast_sales["Sales"] = predicted

# Optimization Section

We use the `ticdat` package to organize the data into "dict-of-dicts". This will facilitate reusing the same code block against different data sources.

In [15]:
from ticdat import TicDatFactory, Slicer
import gurobipy as gu

In [16]:
input_schema = TicDatFactory(products = [["Name"],["Family"]],
                             forecast_sales = [["Product", "Cost Per Unit"],
                                               ["Sales"]],
                             max_promotions = [["Product Family"],["Max Promotions"]]
                             )

In [17]:
dat = input_schema.TicDat(
        products = {'11 Down': 'Clear',
                    'AB Root Beer': 'Dark',
                    'Alpine Stream': 'Clear',
                    'Bright': 'Clear',
                    'Crisp Clear': 'Clear',
                    'DC Kola': 'Dark',
                    'Koala Kola': 'Dark',
                    'Mr. Popper': 'Dark',
                    'Popsi Kola': 'Dark'},
        forecast_sales = forecast_sales.set_index(["Product", "Cost Per Unit"]), 
        max_promotions = {'Dark': 2, 'Clear': 2})
assert (len(dat.forecast_sales) == len(forecast_sales))
assert (set(dat.products) == {pdct for (pdct, cost) in dat.forecast_sales})

In [18]:
normal_price = {pdct:0 for pdct in dat.products}
for pdct, price in dat.forecast_sales:
    normal_price[pdct] = max(normal_price[pdct], price)

It's natural that the predictions don't always imply that cheaper costs will result in more sales. This is more likely for this data set since the training data table was fairly small.

Here is the number of total discount options across all products.

In [19]:
len(dat.forecast_sales) - len(dat.products)

27

Here is the number of meaningful discounts (i.e. discounts that are predicted to increase sales).

In [20]:
number_meaningful_discounts = 0
for (pdct, price), r in dat.forecast_sales.items():
    if (price < normal_price[pdct] and 
        r["Sales"] > dat.forecast_sales[pdct,normal_price[pdct]]["Sales"]):
        number_meaningful_discounts += 1
number_meaningful_discounts

17

In [21]:
def revenue(pdct, price):
    return dat.forecast_sales[pdct, price]["Sales"] * price
def investment(pdct, price):
    return max(0, dat.forecast_sales[pdct, price]["Sales"] * normal_price[pdct] -
                  revenue(pdct, normal_price[pdct]))

In [22]:
mdl = gu.Model("beer promotion")

In [23]:
pdct_price = mdl.addVars(dat.forecast_sales, vtype=gu.GRB.BINARY,name='pdct_price')

In [24]:
mdl.addConstrs((pdct_price.sum(pdct,'*') == 1 for pdct in dat.products), name = "pick_one_price")

{'11 Down': <gurobi.Constr *Awaiting Model Update*>,
 'AB Root Beer': <gurobi.Constr *Awaiting Model Update*>,
 'Alpine Stream': <gurobi.Constr *Awaiting Model Update*>,
 'Bright': <gurobi.Constr *Awaiting Model Update*>,
 'Crisp Clear': <gurobi.Constr *Awaiting Model Update*>,
 'DC Kola': <gurobi.Constr *Awaiting Model Update*>,
 'Koala Kola': <gurobi.Constr *Awaiting Model Update*>,
 'Mr. Popper': <gurobi.Constr *Awaiting Model Update*>,
 'Popsi Kola': <gurobi.Constr *Awaiting Model Update*>}

In [25]:
pf_slice = Slicer((dat.products[pdct]["Family"], pdct, price) for pdct, price in pdct_price)
for pdct_family, r in dat.max_promotions.items():
    mdl.addConstr(gu.quicksum(pdct_price[_pdct, _price]
                              for _pdct_family, _pdct, _price in pf_slice.slice(pdct_family, '*', '*')
                              if _price != normal_price[_pdct]) <= r["Max Promotions"],
                  name = "max_promotions_%s"%pdct_family)

We create dedicated variables for each KPI. The total investment is limited to 500.

In [26]:
max_investment = 500
total_qty = mdl.addVar(name="total_qty")
total_revenue = mdl.addVar(name="total_revenue")
total_investment = mdl.addVar(name="total_investment", ub = max_investment)

In [27]:
mdl.addConstr(total_qty == pdct_price.prod({_:dat.forecast_sales[_]["Sales"] for _ in pdct_price}))
mdl.addConstr(total_revenue == pdct_price.prod({_:revenue(*_) for _ in pdct_price}))
mdl.addConstr(total_investment == pdct_price.prod({_:investment(*_) for _ in pdct_price}))

<gurobi.Constr *Awaiting Model Update*>

In [28]:
mdl.setObjective(total_qty, sense=gu.GRB.MAXIMIZE)
mdl.optimize()

Optimize a model with 14 rows, 39 columns and 155 nonzeros
Variable types: 3 continuous, 36 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 5e+02]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 5881.36
Presolve removed 13 rows and 35 columns
Presolve time: 0.01s
Presolved: 1 rows, 4 columns, 4 nonzeros
Found heuristic solution: objective 6033.6800000
Variable types: 0 continuous, 4 integer (4 binary)

Root relaxation: cutoff, 0 iterations, 0.00 seconds

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 4 (of 4 available processors)

Solution count 3: 6033.68 5881.36 5853.04 
Pool objective bound 6033.68

Optimal solution found (tolerance 1.00e-04)
Best objective 6.033680000000e+03, best bound 6.033680000000e+03, gap 0.0000%


In [29]:
mdl.status == gu.GRB.OPTIMAL

True

Display the KPIs and pricing choices. Observe that the total investment constraint is nearly tight (likely as tight as possible given integrality of variables) and the max promotions constraint is tight for each product family.

In [30]:
(total_qty.X,  total_revenue.X, total_investment.X)

(6033.68, 11506.959925000001, 492.2085250000003)

In [31]:
price_selections = {"Product":[], "Price":[], "Is Discount":[], "Family":[]}
for (pdct, price), var in pdct_price.items():
    if abs(var.X -1) < 0.0001: # i.e. almost one
        price_selections["Product"].append(pdct)
        price_selections["Price"].append(price)
        price_selections["Is Discount"].append(price < normal_price[pdct])
        price_selections["Family"].append(dat.products[pdct]["Family"])

In [32]:
(DataFrame(price_selections).set_index("Product")[["Price", "Is Discount", "Family"]].
 sort_values("Family"))

,Price,Is Discount,Family
Product,,,
11 Down,1.4550,True,Clear
Alpine Stream,2.2275,False,Clear
Crisp Clear,1.4700,False,Clear
Bright,1.2825,True,Clear
DC Kola,1.9150,True,Dark
AB Root Beer,3.8425,False,Dark
Popsi Kola,1.7500,False,Dark
Koala Kola,2.5650,False,Dark
Mr. Popper,2.9000,True,Dark
